In [2]:
import pandas as pd
import geopandas as gpd
import topojson as tp
import re
import warnings

# read in sales
sales_df = pd.read_csv('Data/DeKalbSales.csv', index_col=0)

# convert pandas to geodataframe
sales_df = gpd.GeoDataFrame(
    sales_df,
    geometry=gpd.points_from_xy(
        sales_df['PropertyLongitude'], sales_df['PropertyLatitude']),
    crs="EPSG:4269"
)

# rename columns
sales_df = sales_df.rename(columns={
    'PropertyAddressFull': 'address',
    'RecordingDate': 'sale_date',
    'TransferAmount': 'sale_amt',
    'YearBuilt': 'yr_blt',
    'AreaGross': 'home_size',
    'PropertyLatitude': 'lat',
    'PropertyLongitude': 'long'
})

# read in census tracts
dekalb_ct = gpd.read_file('Data/dekalb_CTs.geojson')

# pare down the dataframe
dekalb_ct = dekalb_ct[[
    'GEOID',
    'Sub_geo',
    'geometry'
]]

# ignore the warnings that come with simplifying geographically
warnings.filterwarnings("ignore", category=RuntimeWarning)

toposimplify = 0.001
dekalb_simp = tp.Topology(dekalb_ct, toposimplify=toposimplify).to_gdf()

# export simplified geometry
dekalb_simp.to_file('Data/dekalb_CTs_simp.geojson')

# spatial join
sales_joined = sales_df.sjoin(dekalb_ct, how="left")

sales_joined = sales_joined.drop(columns='index_right')

# weed-out filters
sales_joined = sales_joined[sales_joined['sale_amt'] >= 20000]
sales_joined = sales_joined[sales_joined['home_size'] >= 75]

# rename sub_geos
sales_joined['Sub_geo'] = sales_joined['Sub_geo'].replace({
    'Lithonia/E.  DeKalb': 'Lithonia/E. DeKalb'
})

# create price / SF
sales_joined['price_sf'] = sales_joined['sale_amt'] / sales_joined['home_size']

# Now that it's joined, drop geometry and lat/longs
sales_joined = sales_joined.drop(columns=[
    'lat',
    'long',
    'geometry'
])

# export
sales_joined.to_csv('Data/DeKalb_20-24.csv', index=False)

# print('export complete!')
print(f'dataframe rows: {sales_joined.shape[0]:,}')
print('------')
sales_joined.head(3)

dataframe rows: 60,957
------


,address,sale_date,year,month,year-month,sale_amt,yr_blt,home_size,GEOID,Sub_geo,price_sf
0,3682 DAISY DR,2020-10-09,2020,10,2020-10,72000.0,1956.0,988.0,13089023108,Central DeKalb,72.874494
1,4301 DOGWOOD FARMS DR,2021-10-13,2021,10,2021-10,264100.0,1983.0,1852.0,13089023426,SW DeKalb,142.602592
2,5350 TERRYTOWN LN,2020-03-06,2020,3,2020-3,123500.0,1999.0,2016.0,13089023432,SW DeKalb,61.259921
